In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb




In [4]:
data=pd.read_csv("/kaggle/input/mydata/MYDATA.csv")

# data preprocessing


In [5]:

# Define the input features
num_features = ['age', 'MarketValue', 'Fee', 'YearOfTranfert', 'Height', 'Squad', 'Appearances', 'Goals', 'Assists', 'OwnGoals', 'SubsON', 'SubsOFF', 'YellowCards', 'SecondYellowCards', 'RedCards', 'PenaltyGoals', 'MinutesPerGoal', 'MinutesPlayed']
cat_features = ['position', 'Country', 'PreviousTeam', 'LeagueOfPreviousTeam', 'CountryOfPreviousTeam', 'NewTeam', 'LeagueOfNewTeam', 'CountryOfNewTeam']


In [6]:
# Define the input features
X = data.drop(['name', 'PPG'], axis=1)
y = data['PPG']

# Encode the categorical features
cat_encoders = {}
for feature in cat_features:
    cat_encoders[feature] = LabelEncoder()
    X[feature] = cat_encoders[feature].fit_transform(X[feature])



In [7]:
X.NewTeam.value_counts()
#data.NewTeam.value_counts()

705     256
1380    191
27      189
60      178
1364    170
       ... 
1265      1
48        1
190       1
1182      1
1110      1
Name: NewTeam, Length: 1499, dtype: int64

In [8]:
X=X.drop('PlaceOfBirth',axis=1)
X=X.drop('Unnamed: 0',axis=1)

In [9]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Evaluate the model
print("XGBoost Regression R^2 Score:", xgb_model.score(X_test, y_test))



XGBoost Regression R^2 Score: 0.3010077537695911


In [10]:
new_data = pd.DataFrame({
    'age': [25],
    'position': ['Central Midfield'],
    'Country': [''],
    'MarketValue': [30],
    'PreviousTeam': [''],
    'LeagueOfPreviousTeam': [''],
    'CountryOfPreviousTeam': [''],
    'Fee': [45],
    'YearOfTranfert': [2024],
    'NewTeam': ['Liverpool'],
    'LeagueOfNewTeam': ['Premier League'],
    'CountryOfNewTeam': ['England'],
    'Height': [180],
    'Squad': [46],
    'Appearances': [46],
    #'PPG': [0.9],
    'Goals': [5],
    'Assists': [7],
    'OwnGoals': [0],
    'SubsON': [0],
    'SubsOFF': [9],
    'YellowCards': [5],
    'SecondYellowCards': [0],
    'RedCards': [0],
    'PenaltyGoals': [8],
    'MinutesPerGoal': [200],
    'MinutesPlayed': [3500]
})
for feature in cat_features:
    cat_encoders[feature] = LabelEncoder()
    new_data[feature] = cat_encoders[feature].fit_transform(new_data[feature])



In [12]:

# Make predictions
xgb_predictions = xgb_model.predict(new_data)


In [ ]:
"""for feature in cat_features:
  encoder = LabelEncoder()
  new_data[feature] = encoder.fit_transform(new_data[feature])

new_player_pred = rf_model.predict(new_data)
print("Predicted player name:", new_player_pred[0])"""

In [21]:
#X_test_encoded = pd.DataFrame(index=X_test.index)
X_test_encoded=X
# Encode the categorical features in X_test
for feature in cat_features:
    encoder = LabelEncoder()
    X_test_encoded[feature] = encoder.fit_transform(X_test_encoded[feature])

# Concatenate the encoded X_test with the numeric X_test
#X_test_all = pd.concat([X_test_encoded, X_test[num_features]], axis=1)

# Get the predicted PPG values for all players in X_test
predicted_ppg = xgb_model.predict(X_test_encoded)

# Find the index of the player in X_test with the closest predicted PPG to new_data
closest_indices = np.argsort(abs(predicted_ppg - xgb_model.predict(new_data)))[:200]
print(closest_indices)


[14384 29722 15971  8962  3737  5374  9303 16876 20780 27073  2691  7749
  1696 11884  2570  9533 22493 25712 25711 20048 21117 23850 19344  8869
 29092   350 25834  3093  7858  9234   801  9119   771   498 19516 17457
  1134  1154  2222 14312  7863 21740 16597  5028  4311  7132  7219  5232
 13708  2146  6380 20939  2030 20983 19285 10235 19404 13133 14026 20955
 28651 17820 10244  9868  2047 19622  8849   993 16565 21697 21642 14208
    69 26134 22583  9286 12488   781  1374   680  7445  2555 22303  2690
  1880  9096 18359 19346  5512  7520 10905  5365 24921    71 25885 12272
  4317 10361    14 15785  6220 19575  3817 12133 20993 12003  7566 10320
  2186 19335  7628 16487  2078 23871 25337  9827 19276  5495 20879  3742
 16705 20381 14461 22759  2425   552  2259 14618 13029 10898 20954 13185
  1634 22532  9675 10813  6500  5517  6827 12672  1830 20964 26748  2001
    38 17637 24170   499 25772 11874 14311 15827 21776 29465  8090  6871
 23581 27210 19927 25746 21332 14393  1615 12400 20

In [22]:
# Get the name of the closest player and print it out as the recommendation
print("Top 3 Recommendations:")
for i in closest_indices:
    if(data.iloc[i]['age']<=25 and data.iloc[i]['position']=='Central Midfield' and data.iloc[i]['YearOfTranfert'] in [2022,2021,2020,2019,2018,2017]):
        print(data.iloc[i]['name'])


Top 3 Recommendations:
Youri Tielemans 
Berkay ?can 


In [ ]:
type(closest_indices)

# The most import features

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Get feature importances from the trained model
feature_importances = rf_model.feature_importances_

# Get names of the features
feature_names = X.columns

# Create a datafr%ame with feature names and importance scores
feature_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})

# Sort the dataframe by importance score in descending order
feature_df = feature_df.sort_values(by='importance', ascending=False)

# Plot the feature importance using a bar chart
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_df)
plt.title('Feature Importance')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()


In [ ]:
print(data.position.value_counts())
print(X.position.value_counts())

# Best Hyperparameters

In [ ]:
"""from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Create a Random Forest Regressor object
rf_model = RandomForestRegressor()

# Define a grid of hyperparameters to search over
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a GridSearchCV object and fit it to the data
grid_search = GridSearchCV(rf_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found by the grid search
print("Best hyperparameters:", grid_search.best_params_)

# Get the best model found by the grid search
best_rf_model = grid_search.best_estimator_
"""